In [1]:
import numpy as np
import pandas as pd

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler


class MyTransformer(BaseEstimator, TransformerMixin):

    def __init__(self):
        self._mean_X = None
        self._std_X = None
        self._mean_y = None
        self._std_y = None

    def fit(self, X: np.array, y: np.array = None):
        if isinstance(X, pd.DataFrame):
            self._mean_X = X.values.mean(axis=0)
            self._std_X = X.values.std(axis=0)
        else:
            self._mean_X = X.mean(axis=0)
            self._std_X = X.std(axis=0)

        if y is None:
            return self

        if isinstance(X, pd.DataFrame):
            self._mean_y = y.values.mean()
            self._std_y = y.values.std()
        else:
            self._mean_y = y.mean()
            self._std_y = y.std()
        return self

    def transform(self, X: np.array, y: np.array = None):
        X_ = (X.copy() - self._mean_X) / self._std_X

        if y is None:
            return X_

        y_ = (y.copy() - self._mean_y) / self._std_y
        return X_, y_

In [2]:
df = pd.DataFrame({
    'a': range(2, 12, 2),
    'b': range(20, 120, 20),
    'c': np.linspace(0, 1, 5),
    'd': np.linspace(-1, 1, 5)
})
df

,a,b,c,d
0,2,20,0.00,-1.0
1,4,40,0.25,-0.5
2,6,60,0.50,0.0
3,8,80,0.75,0.5
4,10,100,1.00,1.0


In [3]:
MyTransformer().fit_transform(df)

,a,b,c,d
0,-1.414214,-1.414214,-1.414214,-1.414214
1,-0.707107,-0.707107,-0.707107,-0.707107
2,0.000000,0.000000,0.000000,0.000000
3,0.707107,0.707107,0.707107,0.707107
4,1.414214,1.414214,1.414214,1.414214


In [4]:
StandardScaler().fit_transform(df)

array([[-1.41421356, -1.41421356, -1.41421356, -1.41421356],
       [-0.70710678, -0.70710678, -0.70710678, -0.70710678],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.70710678,  0.70710678,  0.70710678,  0.70710678],
       [ 1.41421356,  1.41421356,  1.41421356,  1.41421356]])

In [5]:
X_train = np.array([[ 1., -1.,  3.],
                    [ 3.,  0.,  0.],
                    [ 0.,  1., -1.]])

In [6]:
StandardScaler().fit_transform(X_train)

array([[-0.26726124, -1.22474487,  1.37281295],
       [ 1.33630621,  0.        , -0.39223227],
       [-1.06904497,  1.22474487, -0.98058068]])

In [7]:
MyTransformer().fit_transform(X_train)

array([[-0.26726124, -1.22474487,  1.37281295],
       [ 1.33630621,  0.        , -0.39223227],
       [-1.06904497,  1.22474487, -0.98058068]])